# SocialAL Model
### KLS 5.30.19 updated 8.23.19 and 9.11.19

#### Project info: https://osf.io/b48n2/
#### Model modified from :

Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version:

In [ ]:
import sys
print(sys.version)  

## Setup

### Load modules

In [ ]:
import numpy as np
import random
import math

### Initialize free parameters
**a_gain**: gain learning rate (varies 0-1)  
**a_loss**: loss learning rate (varies 0-1)  
**beta**: inverse temperature paramter (varies 0-1)

In [ ]:
a_gain=0.2
a_loss=0.2
beta=1

### Initialize other variables
Three potential trustees: A, B, C [Trustworthy, Neutral, Untrustworthy?]  
**Probs** = Belief about likelihood of reciprocation for each trustee [A, B, C]  
**actionProb** = Probability of a participant investing with the trustee  
**Evs** = Expected value of each potential action towards a partner for [[A1,A2,A3,A4], [B1,B2,B3,B4], [C1,C2,C3,C4]]  
**choice** = The choice (how much to entrust) taken by the participant  
**recip** = Whether or not the trustee reciprocated (y in paper)

In [ ]:
Probs = [0.5, 0.5, 0.5]
actionProb = 0.5
Evs = [[9,9,9,9], [9,9,9,9],[9,9,9,9]]
choice = 2
recip = 0

### Declare constants
Four potential actions: 1, 2, 3, 4  
1 = $\$$0 money is entrusted with trustee  ( $\$$9 retained, $\$$0 potentially reciprocated)  
2 = $\$$3 is entrusted with trustee ( $\$$6 retained, $\$$6 potentially reciprocated)  
3 = $\$$6 is entrusted with trustee ( $\$$3 retained, $\$$12 potentially reciprocated)  
4 = $\$$9 is entrusted with trustee ( $\$$0 retained, $\$$18 potentially reciprocated)

In [ ]:
invest = [0,3,6,9]
retain = [9-x for x in invest]
print ("Retain list is: ", retain)
shared = [2*x for x in invest]
print ("Shared list is: ", shared)

## Define update functions

### Update Values

In [ ]:
def update_value(Prob, EV, choice):
    # here Prob is the probability that the trustee will reciprocate; e.g. Probs[0] for trustee A
    # here EV is the expected value of each action for that trustee; e.g. EVs[0] for trustee A
    # reminder: Python starts indexing at 0 instead of 1
    invest = [0,3,6,9]
    retain = [9-x for x in invest] #print ("Retain list is: ", retain)
    shared = [2*x for x in invest] #print ("Shared list is: ", shared)
  
    EV[choice-1] = retain[choice-1] + Prob*shared[choice-1]
    return EV

### Update probabilities

In [ ]:
def update_prob(recip, Prob, a_gain, a_loss):
    # here Prob is the probability that the trustee will reciprocate; e.g. Probs[0] for trustee A
    gain = max(recip - Prob, 0)
    loss = min(recip - Prob, 0)
    Prob = Prob + a_gain * gain + a_loss * loss
    return Prob

## Test update functions

### Test updating on Trustee A (reciprocates 93% of the time)

In [ ]:
choice = [1,2,3,4,1,2,3,4,1,2,3,4,2,3,4]
random.shuffle(choice)
recip = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,0]
random.shuffle(recip)

for i in range(1,len(choice)):
    Probs[0] = update_prob(recip[i],Probs[0], a_gain, a_loss)
    Evs[0] = update_value(Probs[0], Evs[0], choice[i])
    
print("Ending probability for Trustee A is: ", Probs[0])
print("Ending expected values for Trustee A: ", Evs[0])

### Test updating on Trustee B (reciprocates 60% of the time)

In [ ]:
choice = [1,2,3,4,1,2,3,4,1,2,3,4,2,3,4]
random.shuffle(choice)
recip = [1,1,1,1,1,1,1,1,1,0,0,0,0,0,0]
random.shuffle(recip)

for i in range(1,len(choice)):
    Probs[1] = update_prob(recip[i],Probs[1], a_gain, a_loss)
    Evs[1] = update_value(Probs[1], Evs[1], choice[i])
print("Ending probability for Trustee B is: ", Probs[1])
print("Ending expected values for Trustee B: ", Evs[1])

### Test updating on Trustee C (reciprocates 7% of the time)

In [ ]:
choice = [1,2,3,4,1,2,3,4,1,2,3,4,2,3,4]
random.shuffle(choice)
recip = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
random.shuffle(recip)

for i in range(1,len(choice)):
    Probs[2] = update_prob(recip[i],Probs[2], a_gain, a_loss)
    Evs[2] = update_value(Probs[2], Evs[2], choice[i])
    
print("Ending probability for Trustee B is: ", Probs[2])
print("Ending expected values for Trustee B: ", Evs[2])

## Define action selection functions

### Softmax action selection
_update with code from Jess_

In [ ]:
def get_action_selection_prob(beta, EV, choice):
    # this function calculates the probability of an action using softmax action selection
    # here Prob is the probability that the trustee will reciprocate; e.g. Probs[0] for trustee A
    # here EV is the expected value of each action for that trustee; e.g. EVs[0] for trustee A
    # reminder: Python starts indexing at 0 instead of 1
    numerator = np.exp(beta*EV[choice-1])
    #print "Numerator is: ", numerator
    denominator = np.sum([np.exp(beta*x) for x in EV])
    #print "Denominator is:", denominator
    actionProb = numerator/denominator
    #print ("Action is:", choice)
    #print ("Probability of investing: ", actionProb)
    return actionProb

In [ ]:
def get_action_selection_probs(beta, EV):
    # this function calls the previous function to get all the action probabilities for a particular trustee
    actionProbs = [get_action_selection_prob(beta, EV, x) for x in range(1,5)]
    return actionProbs

In [ ]:
def action_selection(actionProb):
    # this function calls the previous function and uses it to select an action
    cumprob = np.cumsum(actionProb) # cumulative probability 
    num = random.uniform(0,1)# pick a random number between 0 and 1, see where it falls
    if num < cumprob[0]: action = 1
    elif num < cumprob[1]: action = 2
    elif num < cumprob[2]: action = 3
    else: action = 4   
    return action

### Test action selection functions

In [ ]:
#print(beta)
ev = Evs[2] # Change between Evs[0], Evs[1], and Evs[2] to see the probability for different partners
print("Choice 1 probability is: ", get_action_selection_prob(beta, ev, 1))
print("Choice 2 probability is: ", get_action_selection_prob(beta, ev, 2))
print("Choice 3 probability is: ", get_action_selection_prob(beta, ev, 3))
print("Choice 4 probability is: ", get_action_selection_prob(beta, ev, 4))


In [ ]:
probsA = get_action_selection_probs(beta, Evs[0])
print(probsA)
probsB = get_action_selection_probs(beta, Evs[1])
print(probsB)
probsC = get_action_selection_probs(beta, Evs[2])
print(probsC)

In [ ]:
actionA = action_selection(probsA)
print("Action for Person A = ", actionA)
actionB = action_selection(probsB)
print("Action for Person B = ", actionB)
actionC = action_selection(probsC)
print("Action for Person C = ", actionC)